As mentioned [here](http://manuelpm.me/VIMOSlabbook16/log/2017/05/16/cx-359,644,950-and-953.html), in the same folder as [CX359](https://vimos.manuelpm.me/cx359) in the first section of the SEXM there is CX953.

- CX953: 0 to 50. There are at least two sources. One brighter thought the whole spectral range center at around 30. The second one center around 17.

In [2]:
#Import packages
from astropy.io import fits
import os
from stsci.tools import capable
capable.OF_GRAPHICS = False
from pyraf import iraf
import numpy as np
from shutil import copyfile
from astropy.convolution import convolve, Box1DKernel
#Bokeh plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Span, Label, Arrow, NormalHead
from bokeh.models import HoverTool, tools, ColumnDataSource, CustomJS, Slider, BoxAnnotation
output_notebook()

Created directory /root/VIMOSReduced/cx0359/pyraf for cache


Loading BokehJS ...

First we define the parameters to extract the desire spectra:
    - Name of the source
    - Center, low and upper limits of the aperture
    - background sample

In [3]:
sourcename = 'cx953sky'
center = 31
low = -3.7
high = 3.86
b_low = [center-25,center-23]
b_up = [43-center,39-center]
databasegeneral = 'database/apcx359sexm'
filename = 'database/ap'+sourcename
fitsfilename = 'mos_science_sky_extracted_Q3.fits'

First we multiply the 2D spectra times the exposure time

In [4]:
spectrawithsky = fits.open(fitsfilename)
exptime = spectrawithsky[0].header['EXPTIME']
iraf.stsdas()
iraf.images.imutil()
if os.path.exists(sourcename+'.fits'):
    os.remove(sourcename+'.fits')
    

iraf.images.imutil.imarith(fitsfilename,'*', exptime,sourcename)



      +------------------------------------------------------------+
      |       Space Telescope Science Data Analysis System         |    
      |                   STSDAS Version 3.17                      |
      |                                                            |
      |   Space Telescope Science Institute, Baltimore, Maryland   |
      |   Copyright (C) 2014 Association of Universities for       |
      |            Research in Astronomy, Inc.(AURA)               |
      |       See stsdas$copyright.stsdas for terms of use.        |
      |         For help, send e-mail to help@stsci.edu            |
      |                                                            |
      +------------------------------------------------------------+
stsdas/:
 analysis/      examples        hst_calib/      sobsolete/
 contrib/       fitsio/         playpen/        toolbox/
 describe       graphics/       problems


Now we need to change the database of the aperture to define the center. This is the only way I have found to do it automatically. After this define the parameters in the .par file

In [5]:
copyfile(databasegeneral,filename)
#Now read and replace center and upper and lower values
with open(filename) as f:
    for lines in f:
        if 'image' in lines:
                imagename= lines.split()[1]
        if 'center' in lines:
                numerocenter = lines.split()[2]
        if 'low' in lines:
                numerolow = lines.split()[2]
        if 'high' in lines:
                numerohigh = lines.split()[2]
        if 'xmin' in lines:
            bloworiginal  = lines
            blow = lines.replace(lines.split()[1],str(min(b_low)))
        if 'xmax' in lines:
            buporiginal  = lines
            bup = lines.replace(lines.split()[1],str(max(b_up)))
        if 'sample' in lines:
            sampleor = lines
            sampleb = '\t\tsample '+str(min(b_low))+':'+str(max(b_low))+','+str(min(b_up))+':'+str(max(b_up))+'\n'
            break

with open(filename) as f:
    filedata = f.read()

filedata = filedata.replace(imagename,sourcename)
filedata = filedata.replace(numerocenter,str(center))
filedata = filedata.replace(numerolow,str(low))
filedata = filedata.replace(numerohigh,str(high))
filedata = filedata.replace(bloworiginal,blow)
filedata = filedata.replace(buporiginal,bup)
filedata = filedata.replace(sampleor,sampleb)

                                
with open(filename,'w') as f:
    f.write(filedata)

In [6]:
if os.path.exists(sourcename+'.ms.fits'):
    os.remove(sourcename+'.ms.fits')

#Call them 
iraf.noao.twodspec()
iraf.noao.twodspec.apextract()
iraf.noao.twodspec.apextract.setParam('dispaxis','1')
#http://vivaldi.ll.iac.es/sieinvens/siepedia/pmwiki.php?n=HOWTOs.PythonianIRAF
iraf.noao.apextract.apall.setParam('input',sourcename+'.fits')
#try output
iraf.noao.apextract.apall.setParam('output',sourcename+'.ms.fits')
iraf.noao.twodspec.apextract.apall.setParam('recenter','no')
iraf.noao.twodspec.apextract.apall.setParam('resize','no')
iraf.noao.twodspec.apextract.apall.setParam('edit','no')
iraf.noao.twodspec.apextract.apall.setParam('trace','no')
iraf.noao.twodspec.apextract.apall.setParam('interactive','no')
iraf.noao.twodspec.apextract.apall.setParam('apertures','1')
iraf.noao.twodspec.apextract.apall.setParam('find','no')
iraf.noao.twodspec.apextract.apall.setParam('clean','yes')
iraf.noao.twodspec.apextract.apall.setParam('background','average')
iraf.noao.twodspec.apextract.apall.setParam('b_sample','-10,0:0,0')
iraf.noao.apextract.apall.saveParList(filename='uparm/'+sourcename+'.par')
iraf.noao.twodspec.apextract.apall(ParList='uparm/'+sourcename+'.par')

twodspec/:
 apextract/     longslit/
apextract/:
 apall          apedit          apflatten       apnormalize     apscatter
 apdefault@     apfind          apmask          aprecenter      apsum
 apdemos/       apfit           apnoise         apresize        aptrace


In [7]:
#Plotting
##For srfm[0].header["CTYPE1"] = 'LINEAR'
srfm = fits.open(sourcename+'.ms.fits')
#secondstar = srfm[0].data#[0][0]##[0][0] if using clean
secondstar = srfm[0].data[0][0]
xn = srfm[0].header["NAXIS1"]
refx = srfm[0].header["CRVAL1"]
step = srfm[0].header['CD1_1']
cr = srfm[0].header['CRPIX1']
#
xlist = [ refx + step*(i - cr) for i in np.arange(1, len(secondstar)+1) ]
#
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1}, $y)"),
        ]
    )

#Create ColumnDataSource
x = np.array(xlist)
y = np.array(secondstar)
source = ColumnDataSource(data=dict(x=x,y=y))

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra")
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
#Eraaseplot.line(xlist,secondstar)
plot.line('x','y',source=source)

GlyphRenderer(id='9aae32e7-a183-4d8d-a6af-5486558ebab4', ...)

In [8]:
show(plot)

## Normalizing the spectrum



We use the iraf routine continuum:

In [9]:
sample = '5000:9000'
if os.path.exists(sourcename+'cont.fits'):
    os.remove(sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('input',sourcename+'.ms.fits')
iraf.noao.onedspec.continuum.setParam('output',sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('interactive','no')
iraf.noao.onedspec.continuum.setParam('sample',sample)
iraf.noao.onedspec.continuum.saveParList(filename='uparm/cont'+sourcename+'.par')
iraf.noao.onedspec.continuum(ParList='uparm/cont'+sourcename+'.par')

In [10]:
#Plotting
##For srfm[0].header["CTYPE1"] = 'LINEAR'
srfm = fits.open(sourcename+'cont.fits')
#secondstar = srfm[0].data#[0][0]##[0][0] if using clean
secondstar = srfm[0].data[0][0]
xn = srfm[0].header["NAXIS1"]
refx = srfm[0].header["CRVAL1"]
step = srfm[0].header['CD1_1']
cr = srfm[0].header['CRPIX1']
#
xlist = [ refx + step*(i - cr) for i in np.arange(1, len(secondstar)+1) ]
#
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1}, $y)"),
        ]
    )

#Create ColumnDataSource
x = np.array(xlist)
y = np.array(secondstar)
source = ColumnDataSource(data=dict(x=x,y=y))

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra", x_range=(8400,8900), y_range=(0.7,1.1)
              ,active_drag='pan', active_scroll='wheel_zoom'
             )
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())


#Eraaseplot.line(xlist,secondstar)
plot.line('x','y',source=source)


##Emission Lines
class line(object):
    def __init__(self,name):
        self.name = name


diclines = {line('Ca II'):8498,line('Ca II'):8662,line('Ca II'):8541,line('Fe I'):8621,line('Fe I'):8688,
            line('O I'):8446,line('Fe I'):8514,line('Fe I'):8468,
           line('H'+u"\u03B1"):6563,line("(BaII, FeI and CaI)"):6497,
            line('Ca I'):6162,line('Mg I'):8807}

for name, xloc in diclines.iteritems():
    yloc = y[np.where(abs(xloc - x) < 2)[0][0]]
    span = Arrow(end=NormalHead(fill_color='orange', size=10),
             x_start=xloc, y_start = yloc - .15, x_end = xloc, y_end= yloc-.03
            )
    plot.add_layout(span)
    my_label = Label(x=xloc, y=yloc-.03, text=name.name)
    plot.add_layout(my_label)
    
    

show(plot)